# Deploy COA Objects in Transcend:   DBInfo (Demo Project)

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [1]:
%connect name=Transcend

Password ············


Success: 'Transcend' connection established


In [2]:
call adlste_coa.sp_meta_objects_refresh('DBCInfo') 

Success: 0 rows affected

DatabaseName,TableName,TableKind,ObjType,ObjFunction,ProcessName
ADLSTE_COA,coa_dat_DBCinfo,V,coa,dat,DBCinfo
ADLSTE_COA,coat_dat_DBCinfo,T,coat,dat,DBCinfo
ADLSTE_COA,sp_dat_dbcinfo,P,sp,dat,dbcinfo
ADLSTE_COA_Stg,stg_dat_DBCinfo,O,stg,dat,DBCinfo


## Stage Object (adlste_coa_stg)

In [8]:
drop table adlste_coa_stg.stg_dat_dbcinfo;

Success: 22 rows affected

In [9]:
CREATE MULTISET GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_DBCinfo
( Account_Name   VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, Site_ID        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, InfoKey        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, InfoData       VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
) NO PRIMARY INDEX 
ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [10]:
select count(*) from adlste_coa_stg.stg_dat_dbcinfo;

Count(*)
0


## Target Table and View (adlste_coa)

In [11]:
DROP TABLE adlste_coa.coat_dat_DBCinfo;

Success: 26 rows affected

In [12]:
CREATE MULTISET TABLE adlste_coa.coat_dat_DBCinfo
( Account_Name   VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, Site_ID        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, LogDate        DATE Default Current_Date
, InfoKey        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, InfoData       VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
) Primary Index(Site_ID, LogDate);

Success: 0 rows affected

In [13]:
REPLACE VIEW adlste_coa.coa_dat_DBCinfo
as locking row for access
Select * from adlste_coa.coat_dat_DBCinfo;

Success: 6 rows affected

## Stored Proc to manage the merge

In [30]:
REPLACE PROCEDURE adlste_coa.sp_dat_dbcinfo ( spversion VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 

    /* setup audit process */
    
    DECLARE runid  INTEGER;
    DECLARE siteid VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128); 
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    
    SET runid = 0;
    SET tablename = 'adlste_coa.coat_dat_dbcinfo';
    SET callingsp = 'adlste_coa.sp_dat_dbcinfo';
    SET siteid = 'Empty Table';
    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,current_date as StartDate
    ,current_date as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_dbcinfo 
    Group by  1,2,3
    ;
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'START') ;


    DELETE FROM adlste_coa.coat_dat_dbcinfo WHERE (Site_ID, LogDate) IN
    (Select Site_ID, Current_Date From adlste_coa_stg.stg_dat_dbcinfo );
    
    Insert into adlste_coa.coat_dat_dbcinfo 
    Select Account_Name, Site_ID, Current_Date, InfoKey, InfoData 
    from adlste_coa_stg.stg_dat_dbcinfo
    group  by 1,2,3,4,5; 
    
        
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'END') ;

END;

Success: 0 rows affected

In [35]:
insert into adlste_coa_stg.stg_dat_DBCinfo
select 'test account', 'testsiteid123', infokey, infodata from dbc.dbcinfo;

Success: 3 rows affected

In [36]:
call adlste_coa.sp_dat_dbcinfo('1.1')

Success: 0 rows affected

In [37]:
select * from adlste_coa.coa_audit_runlog order by 1

Run_ID,ParentRun_ID,Site_ID,TableName,StartDate,EndDate,CallingSP,SQLVersion,Run_Type,Run_Status,Run_User,Run_TS
1,0,test_SiteID,adlste_coa.coat_audit_run,2020-04-30,2020-05-06,adlste_coa.sp_test,v1.15,Test Run,START,SH186014,2020-05-07 21:03:41
2,1,test_SiteID,adlste_coa.coat_audit_run,2020-04-30,2020-05-06,adlste_coa.sp_test,v1.15,Test Run,CHECKPOINT,SH186014,2020-05-07 21:03:48
3,1,test_SiteID,adlste_coa.coat_audit_run,2020-04-30,2020-05-06,adlste_coa.sp_test,v1.15,Test Run,END,SH186014,2020-05-07 21:03:54
4,0,test_SiteID,adlste_coa.coat_audit_run,2020-04-30,2020-05-06,adlste_coa.sp_test,v1.15,Test Run,START,SH186014,2020-05-07 21:04:32
5,4,test_SiteID,adlste_coa.coat_audit_run,2020-04-30,2020-05-06,adlste_coa.sp_test,v1.15,Test Run,CHECKPOINT,SH186014,2020-05-07 21:04:34
6,4,test_SiteID,adlste_coa.coat_audit_run,2020-04-30,2020-05-06,adlste_coa.sp_test,v1.15,Test Run,END,SH186014,2020-05-07 21:04:37
7,0,,,,,,1.1,normal run,START,SH186014,2020-05-07 21:45:46
8,7,,,,,,1.1,normal run,END,SH186014,2020-05-07 21:45:46
9,0,,adlste_coa.coat_dat_dbcinfo,,,adlste_coa.sp_dat_dbcinfo,1.1,normal run,START,SH186014,2020-05-07 21:48:55
10,9,,adlste_coa.coat_dat_dbcinfo,,,adlste_coa.sp_dat_dbcinfo,1.1,normal run,END,SH186014,2020-05-07 21:48:55


In [38]:
select * from adlste_coa.coa_audit_summary order by 1

Run_ID,Site_ID,LogDate,TableName,CallingSP,SQLVersion,Run_Type,Steps,Errors,Warnings,Start_TS,End_TS
1,test_SiteID,2020-05-07,adlste_coa.coat_audit_run,adlste_coa.sp_test,v1.15,Test Run,3,0,0,2020-05-07 21:03:41,2020-05-07 21:03:54
4,test_SiteID,2020-05-07,adlste_coa.coat_audit_run,adlste_coa.sp_test,v1.15,Test Run,3,0,0,2020-05-07 21:04:32,2020-05-07 21:04:37
7,,2020-05-07,,,1.1,normal run,2,0,0,2020-05-07 21:45:46,2020-05-07 21:45:46
9,,2020-05-07,adlste_coa.coat_dat_dbcinfo,adlste_coa.sp_dat_dbcinfo,1.1,normal run,2,0,0,2020-05-07 21:48:55,2020-05-07 21:48:55
11,Empty Table,2020-05-07,adlste_coa.coat_dat_dbcinfo,adlste_coa.sp_dat_dbcinfo,1.1,normal run,2,0,0,2020-05-07 21:51:40,2020-05-07 21:51:40
13,testsiteid123,2020-05-07,adlste_coa.coat_dat_dbcinfo,adlste_coa.sp_dat_dbcinfo,1.1,normal run,2,0,0,2020-05-07 21:53:28,2020-05-07 21:53:28


Success: 0 rows affected

## Test the Process

In [42]:
insert into adlste_coa_stg.stg_dat_dbcinfo 
Select 'Test_Account',  'Test_SiteID', InfoKey, InfoData from dbc.dbcinfo;

Select * from adlste_coa_stg.stg_dat_dbcinfo 

Success: 3 rows affected

Account_Name,Site_ID,InfoKey,InfoData
Test_Account,Test_SiteID,RELEASE,16.20.32.11
Test_Account,Test_SiteID,VERSION,16.20.32.11
Test_Account,Test_SiteID,LANGUAGE SUPPORT MODE,Standard
Test_Account,Test_SiteID,RELEASE,16.20.32.11
Test_Account,Test_SiteID,VERSION,16.20.32.11
Test_Account,Test_SiteID,LANGUAGE SUPPORT MODE,Standard
Test_Account,Test_SiteID,RELEASE,16.20.32.11
Test_Account,Test_SiteID,VERSION,16.20.32.11
Test_Account,Test_SiteID,LANGUAGE SUPPORT MODE,Standard


In [43]:
call adlste_coa.sp_dat_dbcinfo('1.1');

Success: 0 rows affected

In [44]:
Select * from adlste_coa.coa_dat_dbcinfo 

Account_Name,Site_ID,LogDate,InfoKey,InfoData
Test_Account,Test_SiteID,2020-05-06,LANGUAGE SUPPORT MODE,Standard
Test_Account,Test_SiteID,2020-05-06,RELEASE,16.20.32.11
Test_Account,Test_SiteID,2020-05-06,VERSION,16.20.32.11
